# __PyTorch 03 : Clasificación Binaria__

De forma similar a como lo hicimos en un par de videos atrás, vamos a recrear el problema del __Titanic__, pero usando PyTorch

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [25]:
data = pd.read_pickle("datasets/TitanicTorch.pkl")

In [26]:
data

,Survived,Age,male,Q,S,2,3,Sibsp__1,Sibsp__2,Sibsp__3,Sibsp__4,Sibsp__5,Sibsp__8,Parch__2,Parch__3
0,0,22.000000,1,0,1,0,1,1,0,0,0,0,0,0,1
1,1,38.000000,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1,26.000000,0,0,1,0,1,0,0,0,0,0,0,0,1
3,1,35.000000,0,0,1,0,0,1,0,0,0,0,0,0,0
4,0,35.000000,1,0,1,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,1,0,1,1,0,0,0,0,0,0,0,1,0
887,1,19.000000,0,0,1,0,0,0,0,0,0,0,0,0,0
888,0,29.985856,0,0,1,0,1,1,0,0,0,0,0,0,1
889,1,26.000000,1,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
X = data.drop('Survived',axis=1)
y = data['Survived']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [29]:
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

C:\Users\Marcelojtc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Marcelojtc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Marcelojtc\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


## Convirtiendo a Tensors

In [30]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

## __Diseñando la Red__

In [31]:
X_train_tensor.shape

torch.Size([705, 14])

In [10]:
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 100)  
        self.fc2 = nn.Linear(100, 300)
        self.fc3 = nn.Linear(300, 200)
        self.fc4 = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# __Instanciando El Modelo__

In [35]:
model = BinaryClassificationModel()

In [36]:
perdida = nn.BCELoss()

In [37]:
optimizador = optim.Adam(model.parameters(), lr=0.001)

# __Entrenando El Modelo__

In [38]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizador.zero_grad()
    outputs = model(X_train_tensor)
    loss = perdida(outputs, y_train_tensor)
    loss.backward()
    optimizadr.step()
    
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        predictions = (outputs >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")

ValueError: Using a target size (torch.Size([705])) that is different to the input size (torch.Size([705, 1])) is deprecated. Please ensure they have the same size.

# __Resolviendo el Error__

In [40]:
y_train_tensor.shape

torch.Size([705])

In [41]:
y_train_tensor = y_train_tensor.unsqueeze(dim = 1)
y_test_tensor = y_test_tensor.unsqueeze(dim = 1)

In [42]:
y_train_tensor.shape

torch.Size([705, 1])

# __Volviendo A Entrenar__

In [43]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizador.zero_grad()
    outputs = model(X_train_tensor)
    loss = perdida(outputs, y_train_tensor)
    loss.backward()
    optimizador.step()
    
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        predictions = (outputs >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")

Epoch [1/50], Loss: 0.6905, Accuracy: 61.02%
Epoch [2/50], Loss: 0.6791, Accuracy: 61.02%
Epoch [3/50], Loss: 0.6693, Accuracy: 61.02%
Epoch [4/50], Loss: 0.6605, Accuracy: 61.02%
Epoch [5/50], Loss: 0.6520, Accuracy: 61.02%
Epoch [6/50], Loss: 0.6432, Accuracy: 61.02%
Epoch [7/50], Loss: 0.6341, Accuracy: 61.02%
Epoch [8/50], Loss: 0.6243, Accuracy: 61.02%
Epoch [9/50], Loss: 0.6137, Accuracy: 61.02%
Epoch [10/50], Loss: 0.6025, Accuracy: 63.84%
Epoch [11/50], Loss: 0.5906, Accuracy: 66.10%
Epoch [12/50], Loss: 0.5783, Accuracy: 70.06%
Epoch [13/50], Loss: 0.5656, Accuracy: 71.19%
Epoch [14/50], Loss: 0.5527, Accuracy: 73.45%
Epoch [15/50], Loss: 0.5396, Accuracy: 76.27%
Epoch [16/50], Loss: 0.5266, Accuracy: 76.84%
Epoch [17/50], Loss: 0.5141, Accuracy: 75.14%
Epoch [18/50], Loss: 0.5021, Accuracy: 75.14%
Epoch [19/50], Loss: 0.4908, Accuracy: 74.58%
Epoch [20/50], Loss: 0.4803, Accuracy: 75.71%
Epoch [21/50], Loss: 0.4707, Accuracy: 76.27%
Epoch [22/50], Loss: 0.4623, Accuracy: 76.2

## __Predicciones__

In [44]:
predicciones = model(X_test_tensor)

In [46]:
predicciones.shape

torch.Size([177, 1])

In [47]:
predicciones = torch.round(predicciones)

## __Evaluaciones__

In [49]:
from sklearn.metrics import classification_report

In [50]:
print(classification_report(y_test_tensor,predicciones))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

## __¿Qué Ocurrió? : El Concepto De Grafo__

* Un grafo de computación es como un plano o un mapa que describe cómo los datos fluyen a través de una serie de operaciones.
* En PyTorch, este grafo lleva un registro de todas las operaciones que se aplican a los tensores (la estructura de datos básica en PyTorch).
* Este grafo es esencial para la diferenciación automática, que permite a PyTorch calcular los gradientes necesarios para optimizar redes neuronales.

Recordemos que estamos trabajando con __Tensors__, no con __Numpy Arrays__. Internamente, PyTorch trata de convertirlo a Numpy, pero no puede porque son parte del grafo y para PyTorch esto es una especie de interrupción a su seguimiento.  

Simplemente, debemos utilizar __detach__, para poder crear un nuevo tensor que no es parte del grafo y así poder utilizarlo

In [53]:
predicciones[:5]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SliceBackward0>)

In [54]:
predicciones.detach().numpy()[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [55]:
print(classification_report(y_test_tensor,predicciones.detach().numpy()))

              precision    recall  f1-score   support

         0.0       0.74      0.89      0.81       108
         1.0       0.75      0.52      0.62        69

    accuracy                           0.75       177
   macro avg       0.75      0.71      0.71       177
weighted avg       0.75      0.75      0.73       177



# __Siguiendo El Modelo Paso a Paso__

In [ ]:
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 100)  # Adjust input size
        self.fc2 = nn.Linear(100, 300)
        self.fc3 = nn.Linear(300, 200)
        self.fc4 = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

In [56]:
capa1 = nn.Linear(X_train.shape[1], 100)
capa2 = nn.Linear(100, 300)
capa3 = nn.Linear(300, 200)
capa4 = nn.Linear(200, 1)
sigmoidFinal = nn.Sigmoid()

In [57]:
X_train_scaled

array([[0.7260274 , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.39706652, 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.39706652, 1.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.01369863, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.39706652, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.43835616, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [58]:
capa1(X_train_scaled)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

### __Recordar : Estamos Trabajando Con Tensors__

In [59]:
outputs1 = capa1(X_train_tensor)
outputs1

tensor([[-0.1741, -0.0732, -0.0329,  ...,  0.1878, -0.4276,  0.0328],
        [ 0.0978,  0.3097,  0.6096,  ..., -0.1553,  0.1407,  0.0484],
        [ 0.3174,  0.2746,  0.3649,  ..., -0.2930,  0.5226,  0.3182],
        ...,
        [-0.2174,  0.0479,  0.0874,  ...,  0.0035, -0.5874,  0.1191],
        [ 0.0255, -0.1236, -0.3003,  ..., -0.0910,  0.2055,  0.1872],
        [ 0.0280, -0.1306, -0.3073,  ..., -0.0804,  0.2148,  0.1822]],
       grad_fn=<AddmmBackward0>)

In [61]:
X_train_tensor.shape

torch.Size([705, 14])

In [60]:
outputs1.shape

torch.Size([705, 100])

## __No Olvidar La Activación ReLU__

In [62]:
outputs1 = torch.relu(outputs1)
outputs1

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1878, 0.0000, 0.0328],
        [0.0978, 0.3097, 0.6096,  ..., 0.0000, 0.1407, 0.0484],
        [0.3174, 0.2746, 0.3649,  ..., 0.0000, 0.5226, 0.3182],
        ...,
        [0.0000, 0.0479, 0.0874,  ..., 0.0035, 0.0000, 0.1191],
        [0.0255, 0.0000, 0.0000,  ..., 0.0000, 0.2055, 0.1872],
        [0.0280, 0.0000, 0.0000,  ..., 0.0000, 0.2148, 0.1822]],
       grad_fn=<ReluBackward0>)

In [63]:
outputs1.shape

torch.Size([705, 100])

## __Seguimos Con La Capa 2 Y Aplicamos ReLU__

In [64]:
outputs2 = torch.relu(capa2(outputs1))
outputs2

tensor([[0.0000, 0.0000, 0.1758,  ..., 0.2485, 0.0000, 0.3391],
        [0.0000, 0.2453, 0.0579,  ..., 0.0045, 0.0000, 0.0000],
        [0.1682, 0.1132, 0.0739,  ..., 0.0993, 0.0000, 0.0260],
        ...,
        [0.0000, 0.0256, 0.1303,  ..., 0.2637, 0.0000, 0.3354],
        [0.0265, 0.0728, 0.0950,  ..., 0.2927, 0.0000, 0.2167],
        [0.0231, 0.0724, 0.0946,  ..., 0.2943, 0.0000, 0.2194]],
       grad_fn=<ReluBackward0>)

In [65]:
outputs2.shape

torch.Size([705, 300])

## __Seguimos Con La Capa 3 Y Aplicamos ReLU__

In [66]:
outputs3 = torch.relu(capa3(outputs2))
outputs3

tensor([[0.0975, 0.0000, 0.0324,  ..., 0.1700, 0.0000, 0.0046],
        [0.0000, 0.0000, 0.0316,  ..., 0.1263, 0.0599, 0.0132],
        [0.0080, 0.0000, 0.0467,  ..., 0.1385, 0.0161, 0.0602],
        ...,
        [0.0972, 0.0000, 0.0219,  ..., 0.1780, 0.0000, 0.0034],
        [0.0706, 0.0000, 0.0281,  ..., 0.1234, 0.0051, 0.0067],
        [0.0715, 0.0000, 0.0285,  ..., 0.1235, 0.0053, 0.0062]],
       grad_fn=<ReluBackward0>)

In [67]:
outputs3.shape

torch.Size([705, 200])

## __Capa 4 Y Activación Final__

In [68]:
outputs4 = capa4(outputs3)
outputs4.shape

torch.Size([705, 1])

In [69]:
outputs4[:5]

tensor([[-0.0317],
        [-0.0340],
        [-0.0334],
        [-0.0434],
        [-0.0332]], grad_fn=<SliceBackward0>)

In [70]:
final_outputs = sigmoidFinal(outputs4)
final_outputs.shape

torch.Size([705, 1])

In [71]:
final_outputs[:5]

tensor([[0.4921],
        [0.4915],
        [0.4916],
        [0.4892],
        [0.4917]], grad_fn=<SliceBackward0>)

## __Calculando La Pérdida__

In [72]:
perdida(final_outputs, y_train_tensor).

tensor(0.6885, grad_fn=<BinaryCrossEntropyBackward0>)